In [ ]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

In [2]:
cur_dir = os.getcwd()
cur_dir

'/home/hadoop/MSD_Project/DataEngineeringProject/Basic analysis hdf5_to_rdd'

Spark Configuration

In [2]:
# Spark Session - 2 cores
spark_session = SparkSession.builder \
    .appName("Test_1File") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 15:33:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
/home/hadoop/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


File Path

In [3]:
#file_roots = ['hdfs://130.238.29.183:9000//input/millionsongsubset.tar.gz']
#directory_path = "data/B/A/A"
#directory_path = "/home/hadoop/MillionSongSubset/A/A/A"
directory_path = "/MillionSongSubset/A/A/A"

In [ ]:
text = spark_session.read.text("")

In [11]:
textFile = spark_session.read.text("test_hdfs.txt")

24/03/07 21:46:46 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/07 21:46:46 INFO SharedState: Warehouse path is 'file:/home/hadoop/MSD_Project/DataEngineeringProject/Basic%20analysis%20hdf5_to_rdd/spark-warehouse'.
24/03/07 21:46:46 INFO ServerInfo: Adding filter to /SQL: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/07 21:46:46 INFO ServerInfo: Adding filter to /SQL/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/07 21:46:46 INFO ServerInfo: Adding filter to /SQL/execution: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/07 21:46:46 INFO ServerInfo: Adding filter to /SQL/execution/json: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/07 21:46:46 INFO ServerInfo: Adding filter to /static/sql: org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter
24/03/07 21:46:47 INFO InMemoryFileIndex: It took 55 ms to list leaf files for 1 paths.


In [13]:
textFile.show()

24/03/07 21:47:15 INFO FileSourceStrategy: Pushed Filters: 
24/03/07 21:47:15 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/07 21:47:15 INFO CodeGenerator: Code generated in 296.142606 ms
24/03/07 21:47:15 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 353.5 KiB, free 366.0 MiB)
24/03/07 21:47:15 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 35.7 KiB, free 365.9 MiB)
24/03/07 21:47:15 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on master-node:44843 (size: 35.7 KiB, free: 366.3 MiB)
24/03/07 21:47:15 INFO SparkContext: Created broadcast 0 from showString at NativeMethodAccessorImpl.java:0
24/03/07 21:47:15 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/03/07 21:47:16 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/07 21:47:16 INFO DAGScheduler: Got job 0 (showString at Nativ

+-------------+
|        value|
+-------------+
|test for hdfs|
+-------------+



24/03/07 21:47:20 INFO CodeGenerator: Code generated in 21.868835 ms


# 1) Saving the paths to the HDF5 files

In [4]:
# GET .H5 FILENAMES
h5_file_paths = []
for root, _, files in os.walk(directory_path):
    for name in files:
        h5_file_paths.append(os.path.join(root, name))

print('{} filenames gathered (PYTHON)'.format(len(h5_file_paths)))

0 filenames gathered (PYTHON)


# 2) List h5_file_paths is converted to RDD

In [8]:
file_paths = spark_context.parallelize(h5_file_paths)

In [9]:
file_first = file_paths.first()

24/03/07 21:27:36 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/07 21:27:36 INFO DAGScheduler: Got job 0 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/07 21:27:36 INFO DAGScheduler: Final stage: ResultStage 0 (runJob at PythonRDD.scala:181)
24/03/07 21:27:36 INFO DAGScheduler: Parents of final stage: List()
24/03/07 21:27:36 INFO DAGScheduler: Missing parents: List()
24/03/07 21:27:36 INFO DAGScheduler: Submitting ResultStage 0 (PythonRDD[1] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/07 21:27:36 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 5.9 KiB, free 366.3 MiB)
24/03/07 21:27:36 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 3.8 KiB, free 366.3 MiB)
24/03/07 21:27:36 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on master-node:41003 (size: 3.8 KiB, free: 366.3 MiB)
24/03/07 21:27:36 INFO SparkContext: Created broadcast 0 from broadcast at

ValueError: RDD is empty

# 3) H5Py file object file creation from the H5 file 

In [19]:
file = h5py.File(file_first)

In [20]:
file.keys()

<KeysViewHDF5 ['analysis', 'metadata', 'musicbrainz']>

In [22]:
file['analysis']

<HDF5 group "/analysis" (16 members)>

In [23]:
print(list(file['analysis']))
print(list(file['metadata']))
print(list(file['musicbrainz']))

['bars_confidence', 'bars_start', 'beats_confidence', 'beats_start', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_pitches', 'segments_start', 'segments_timbre', 'songs', 'tatums_confidence', 'tatums_start']
['artist_terms', 'artist_terms_freq', 'artist_terms_weight', 'similar_artists', 'songs']
['artist_mbtags', 'artist_mbtags_count', 'songs']


In [24]:
print(list(file['analysis']['songs']))
print(list(file['metadata']['songs']))
print(list(file['musicbrainz']['songs']))

[(22050, b'9938b27e93cf7a47816625bc53ab7920', 0., 210.75546, 0.153, 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.401, -4.188, 0, 0.471, 203.122, 150.03, 4, 0.57, b'TRBAASG128F92FBB7C')]
[(b'', 457243, 0.45537186, 0.3138666, b'AROLMXS1241B9C6915', nan, b'', nan, b'', b'An Cafe', -1, b'', 0, 0, b'Magnya Carta', 395941, nan, b'SOPIHNL12A8C13B28D', b'Pipopapo Telpathy', 4393959)]
[(0, 0)]


In [28]:
song = file['analysis']['songs']
song

<HDF5 dataset "songs": shape (1,), type "|V220">

In [29]:
np_song = np.array(song[0])
np_song

array((22050, b'9938b27e93cf7a47816625bc53ab7920', 0., 210.75546, 0.153, 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.401, -4.188, 0, 0.471, 203.122, 150.03, 4, 0.57, b'TRBAASG128F92FBB7C'),
      dtype=[('analysis_sample_rate', '<i4'), ('audio_md5', 'S32'), ('danceability', '<f8'), ('duration', '<f8'), ('end_of_fade_in', '<f8'), ('energy', '<f8'), ('idx_bars_confidence', '<i4'), ('idx_bars_start', '<i4'), ('idx_beats_confidence', '<i4'), ('idx_beats_start', '<i4'), ('idx_sections_confidence', '<i4'), ('idx_sections_start', '<i4'), ('idx_segments_confidence', '<i4'), ('idx_segments_loudness_max', '<i4'), ('idx_segments_loudness_max_time', '<i4'), ('idx_segments_loudness_start', '<i4'), ('idx_segments_pitches', '<i4'), ('idx_segments_start', '<i4'), ('idx_segments_timbre', '<i4'), ('idx_tatums_confidence', '<i4'), ('idx_tatums_start', '<i4'), ('key', '<i4'), ('key_confidence', '<f8'), ('loudness', '<f8'), ('mode', '<i4'), ('mode_confidence', '<f8'), ('start_of_fade_out', '<f8')

In [30]:
selected_features = np_song[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
selected_features

array([(0., 210.75546, 150.03, 0., -4.188)],
      dtype={'names': ['danceability', 'duration', 'tempo', 'energy', 'loudness'], 'formats': ['<f8', '<f8', '<f8', '<f8', '<f8'], 'offsets': [36, 44, 168, 60, 140], 'itemsize': 220})

In [31]:
rdd_data = spark_context.parallelize([tuple(selected_features)])
rdd_data.first()

24/03/01 10:37:58 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/01 10:37:58 INFO DAGScheduler: Got job 1 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/01 10:37:58 INFO DAGScheduler: Final stage: ResultStage 1 (runJob at PythonRDD.scala:181)
24/03/01 10:37:58 INFO DAGScheduler: Parents of final stage: List()
24/03/01 10:37:58 INFO DAGScheduler: Missing parents: List()
24/03/01 10:37:59 INFO DAGScheduler: Submitting ResultStage 1 (PythonRDD[3] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/01 10:37:59 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 5.8 KiB, free 413.9 MiB)
24/03/01 10:37:59 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 3.8 KiB, free 413.9 MiB)
24/03/01 10:37:59 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on host-192-168-2-34-de1:46587 (size: 3.8 KiB, free: 413.9 MiB)
24/03/01 10:37:59 INFO SparkContext: Created broadcast 1 from br

((0., 210.75546, 150.03, 0., -4.188),)

# Convertion to pandas

In [35]:
data_df = pd.DataFrame(selected_features)
data_df

,danceability,duration,tempo,energy,loudness
0,0.0,210.75546,150.03,0.0,-4.188


# Convertion to Spark - Dataframe

In [36]:
columns = ['danceability', 'duration', 'tempo', 'energy', 'loudness']
df = sqlContext.createDataFrame(data_df)

24/03/01 10:40:20 INFO BlockManagerInfo: Removed broadcast_2_piece0 on host-192-168-2-34-de1:46587 in memory (size: 3.8 KiB, free: 413.9 MiB)
24/03/01 10:40:20 INFO BlockManagerInfo: Removed broadcast_1_piece0 on host-192-168-2-34-de1:46587 in memory (size: 3.8 KiB, free: 413.9 MiB)
24/03/01 10:40:20 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/01 10:40:20 INFO SharedState: Warehouse path is 'file:/home/ubuntu/dataeng1/Project/spark-warehouse'.


# Calculation of statistics

In [37]:
summary = df.select(
    mean('danceability').alias('mean_danceability'),
    min('danceability').alias('min_danceability'),
    max('danceability').alias('max_danceability'),
    stddev('danceability').alias('stddev_danceability'),
    
    mean('duration').alias('mean_duration'),
    min('duration').alias('min_duration'),
    max('duration').alias('max_duration'),
    stddev('duration').alias('stddev_duration'),
    
    mean('tempo').alias('mean_tempo'),
    min('tempo').alias('min_tempo'),
    max('tempo').alias('max_tempo'),
    stddev('tempo').alias('stddev_tempo'),
    
    mean('energy').alias('mean_energy'),
    min('energy').alias('min_energy'),
    max('energy').alias('max_energy'),
    stddev('energy').alias('stddev_energy'),
    
    mean('loudness').alias('mean_loudness'),
    min('loudness').alias('min_loudness'),
    max('loudness').alias('max_loudness'),
    stddev('loudness').alias('stddev_loudness')
).collect()

24/03/01 10:40:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/03/01 10:40:32 INFO CodeGenerator: Code generated in 1103.189511 ms
24/03/01 10:40:32 INFO DAGScheduler: Registering RDD 11 (collect at /tmp/ipykernel_52445/3345987588.py:1) as input to shuffle 0
24/03/01 10:40:32 INFO DAGScheduler: Got map stage job 3 (collect at /tmp/ipykernel_52445/3345987588.py:1) with 2 output partitions
24/03/01 10:40:32 INFO DAGScheduler: Final stage: ShuffleMapStage 3 (collect at /tmp/ipykernel_52445/3345987588.py:1)
24/03/01 10:40:32 INFO DAGScheduler: Parents of final stage: List()
24/03/01 10:40:32 INFO DAGScheduler: Missing parents: List()
24/03/01 10:40:32 INFO DAGScheduler: Submitting ShuffleMapStage 3 (MapPartitionsRDD[11] at collect at /tmp/ipykernel_52445/3345987588.py:1), which has no missing parents
24/03/01 10:40:32 INFO MemoryStore: Block broadcast_3 stored as

In [52]:
# Anzahl der Songs im DataFrame
num_songs = df.count()
print(f"\nNumber of songs contributing to the calculation: {num_songs}")
print()

# Ergebnisse ausgeben
for i, row in enumerate(summary, start=1):
    for col in row.asDict():
        print(f"{col}:\t{row[col]}")

24/03/01 10:48:16 INFO DAGScheduler: Registering RDD 76 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 13
24/03/01 10:48:16 INFO DAGScheduler: Got map stage job 29 (count at NativeMethodAccessorImpl.java:0) with 2 output partitions
24/03/01 10:48:16 INFO DAGScheduler: Final stage: ShuffleMapStage 42 (count at NativeMethodAccessorImpl.java:0)
24/03/01 10:48:16 INFO DAGScheduler: Parents of final stage: List()
24/03/01 10:48:16 INFO DAGScheduler: Missing parents: List()
24/03/01 10:48:16 INFO DAGScheduler: Submitting ShuffleMapStage 42 (MapPartitionsRDD[76] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/01 10:48:16 INFO MemoryStore: Block broadcast_29 stored as values in memory (estimated size 15.9 KiB, free 413.8 MiB)
24/03/01 10:48:16 INFO MemoryStore: Block broadcast_29_piece0 stored as bytes in memory (estimated size 8.3 KiB, free 413.8 MiB)
24/03/01 10:48:16 INFO BlockManagerInfo: Added broadcast_29_piece0 in memory on host-192-168-2


Number of songs contributing to the calculation: 1

mean_danceability:	0.0
min_danceability:	0.0
max_danceability:	0.0
stddev_danceability:	None
mean_duration:	210.75546
min_duration:	210.75546
max_duration:	210.75546
stddev_duration:	None
mean_tempo:	150.03
min_tempo:	150.03
max_tempo:	150.03
stddev_tempo:	None
mean_energy:	0.0
min_energy:	0.0
max_energy:	0.0
stddev_energy:	None
mean_loudness:	-4.188
min_loudness:	-4.188
max_loudness:	-4.188
stddev_loudness:	None


24/03/01 10:48:17 INFO ShuffleBlockFetcherIterator: Getting 2 (120.0 B) non-empty blocks including 2 (120.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
24/03/01 10:48:17 INFO ShuffleBlockFetcherIterator: Started 0 remote fetches in 4 ms
24/03/01 10:48:17 INFO Executor: Finished task 0.0 in stage 44.0 (TID 44). 4038 bytes result sent to driver
24/03/01 10:48:17 INFO TaskSetManager: Finished task 0.0 in stage 44.0 (TID 44) in 47 ms on host-192-168-2-34-de1 (executor driver) (1/1)
24/03/01 10:48:17 INFO DAGScheduler: ResultStage 44 (count at NativeMethodAccessorImpl.java:0) finished in 0.064 s
24/03/01 10:48:17 INFO DAGScheduler: Job 30 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/01 10:48:17 INFO TaskSchedulerImpl: Removed TaskSet 44.0, whose tasks have all completed, from pool 
24/03/01 10:48:17 INFO TaskSchedulerImpl: Killing all running tasks in stage 44: Stage finished
24/03/01 10:48:17 INFO DAGSche

# ----------------- Other contents of the song data ------------------------------------

In [54]:
dataset_value = file['metadata']['artist_terms'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[b'dance rock']
Length: 1


In [55]:
dataset_value = file['metadata']['artist_terms_freq'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[1.]
Length: 1


In [56]:
dataset_value = file['metadata']['similar_artists'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[b'AR7XLQX1187B99A7EA' b'AREVXEQ12086C1134E' b'AR47RE6119B8668F53'
 b'ARQ1Q9X1187FB47A97' b'ART5G6A1187FB501EF' b'AR8B0LB1187B9B9A00'
 b'ARSFIZV12509415792' b'ARFBMCW122988FF6AB' b'ARQYLGZ12AF7DA825B'
 b'ARN0OPB1187FB3BB94' b'ARYQSVT123526A064F' b'AR83XCX1187B9AEED2'
 b'ARJUPUZ12802FDEAA6' b'ARGUT1W1187FB5AF5B' b'ARMMNNW12AA61C90AA'
 b'ARZISWL11F4C844108' b'ARJAAWU1187FB55EAC' b'ARJF4HD1187FB3731B'
 b'ARAVUNG1269FB31D22' b'ARHFRME122988FCF98' b'AR9NHBO119B86678BB'
 b'ARF08S41187B992B72' b'ARKMR711187FB3CA5A' b'AR4M5UL11C8A4160FE'
 b'AR9SU0S1187FB53D0F' b'ARHV8FW1187FB452AA' b'ARXPZTL1257509E207'
 b'ARAAC0U1187FB59BA9' b'AR8EE6Q1187B9978E7' b'ARXTEEE1269FCD7AF6'
 b'ARD8L7F1187FB430B3' b'ARVPUOZ12AF7D8E25B' b'ARAGYDJ11F50C4DE49'
 b'ARQHRLD1187FB38A89' b'ARGAT011187FB4FB90' b'ARLBNSN122988ED42B'
 b'ARRQ2TB1187B9A9890' b'AR85JLJ1187B98A3C4' b'ARXZTVV11F43A69F15'
 b'AR5Y3W01187FB47D6F' b'AR324A81187B9A8EDF' b'AR3U9B41187FB3B1A9'
 b'ARZA3WL11C8A42C824' b'AR19UAA1187FB3BDFA' b'ARTJ7RD1187FB4A

In [57]:
dataset_value = file['analysis']['songs'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[(22050, b'9938b27e93cf7a47816625bc53ab7920', 0., 210.75546, 0.153, 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.401, -4.188, 0, 0.471, 203.122, 150.03, 4, 0.57, b'TRBAASG128F92FBB7C')]
Length: 1


In [58]:
dataset_value = file['metadata']['songs'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[(b'', 457243, 0.45537186, 0.3138666, b'AROLMXS1241B9C6915', nan, b'', nan, b'', b'An Cafe', -1, b'', 0, 0, b'Magnya Carta', 395941, nan, b'SOPIHNL12A8C13B28D', b'Pipopapo Telpathy', 4393959)]
Length: 1


In [59]:
dataset_value = file['musicbrainz']['songs'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[(0, 0)]
Length: 1


In [60]:
dataset_value = file['analysis']['bars_confidence'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[0.002 0.153 0.016 0.199 0.174 0.018 0.115 0.3   0.018 0.12  0.582 0.036
 0.005 0.031 0.101 0.004 0.007 0.352 0.324 0.07  0.085 0.273 0.025 0.273
 0.047 0.102 0.383 0.04  0.002 0.242 0.056 0.038 0.082 0.218 0.078 0.005
 0.27  0.005 0.109 0.05  0.166 0.038 0.009 0.301 0.133 0.082 0.073 0.026
 0.022 0.178 0.098 0.428 0.276 0.171 0.63  0.003 0.106 0.131 0.518 0.026
 0.026 0.208 0.059 0.065 0.01  0.085 0.083 0.244 0.165 0.252 0.004 0.18
 0.703 0.207 0.425 0.034 0.06  0.003 0.01  0.07  0.019 0.047 0.045 0.02
 0.151 0.006 0.078 0.043 0.008 0.153 0.001 0.14  0.02  0.009 0.011 0.036
 0.033 0.014 0.155 0.011 0.376 0.251 0.106 0.007 0.071 0.184 0.018 0.238
 0.005 0.001 0.069 0.113 0.137 0.001 0.028 0.211 0.03  0.023 0.084 0.033
 0.054 0.05  0.171 0.008 0.026 0.006 0.024 0.   ]
Length: 128


In [61]:
dataset_value = file['analysis']['bars_start'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[  0.60646   2.2007    3.80092   5.39466   6.98712   8.58864  10.19087
  11.78712  13.38832  14.98951  16.58824  18.18993  19.78535  21.384
  22.98502  24.58846  26.1942   27.80012  29.40439  31.00516  32.60641
  34.2086   35.81714  37.41738  39.00284  40.5968   42.20063  43.80482
  45.40648  47.00735  48.60078  50.20179  51.79572  53.39508  54.99478
  56.597    58.20143  59.80227  61.39652  62.99814  64.59227  66.18748
  67.78402  69.38897  70.99026  72.59427  74.19724  75.79915  77.39534
  79.00029  80.60989  82.21514  83.816    85.41266  87.01549  88.61081
  90.19729  91.79204  93.39821  95.00438  96.60963  98.2031   99.7976
 101.39496 102.99886 104.59913 106.19286 107.79878 109.40192 111.00497
 112.60384 114.19554 115.79468 117.39894 119.00121 120.60796 122.2106
 123.81686 125.41215 126.99845 128.61968 130.00943 131.60719 133.21429
 134.8268  136.41299 138.00936 139.60249 141.19589 142.78573 144.38555
 145.99335 147.60352 149.20855 150.80905 152.40916 154.00758 155.60812
 157.20748

In [62]:
dataset_value = file['analysis']['songs'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[(22050, b'9938b27e93cf7a47816625bc53ab7920', 0., 210.75546, 0.153, 0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.401, -4.188, 0, 0.471, 203.122, 150.03, 4, 0.57, b'TRBAASG128F92FBB7C')]
Length: 1


In [63]:
dataset_value = file['metadata']['songs'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[(b'', 457243, 0.45537186, 0.3138666, b'AROLMXS1241B9C6915', nan, b'', nan, b'', b'An Cafe', -1, b'', 0, 0, b'Magnya Carta', 395941, nan, b'SOPIHNL12A8C13B28D', b'Pipopapo Telpathy', 4393959)]
Length: 1


In [64]:
dataset_value = file['analysis']['segments_start'][:]
print(dataset_value)
print("Length:", len(dataset_value))

[0.0000000e+00 1.5329000e-01 3.8032000e-01 5.7778000e-01 9.8408000e-01
 1.3727900e+00 1.6226800e+00 1.7735100e+00 2.1789600e+00 2.5512000e+00
 2.6499300e+00 2.7487100e+00 3.1318800e+00 3.5498000e+00 3.7532000e+00
 4.3390900e+00 4.5597300e+00 4.9949700e+00 5.3550600e+00 6.1331500e+00
 6.2956900e+00 6.5451700e+00 6.9266700e+00 7.3518800e+00 7.7409500e+00
 8.1355100e+00 8.5398600e+00 8.9429900e+00 9.2328800e+00 9.3430800e+00
 9.7432200e+00 1.0149930e+01 1.0567660e+01 1.0939410e+01 1.1352150e+01
 1.1571750e+01 1.1676050e+01 1.1873970e+01 1.1973740e+01 1.2267350e+01
 1.2366120e+01 1.2657780e+01 1.3314510e+01 1.3841900e+01 1.3964170e+01
 1.4358870e+01 1.4521180e+01 1.4764900e+01 1.5159730e+01 1.5426800e+01
 1.5921090e+01 1.6337870e+01 1.6540410e+01 1.6732740e+01 1.7127890e+01
 1.7341320e+01 1.7539680e+01 1.7928300e+01 1.8341000e+01 1.8451160e+01
 1.8636830e+01 1.8950520e+01 1.9147710e+01 1.9751790e+01 1.9937410e+01
 2.0343170e+01 2.0552380e+01 2.0744040e+01 2.1121500e+01 2.1342220e+01
 2.153

In [5]:
# release the cores
spark_context.stop()

24/03/14 15:36:53 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/03/14 15:36:53 INFO SparkUI: Stopped Spark web UI at http://master-node:4040
24/03/14 15:36:53 INFO YarnClientSchedulerBackend: Interrupting monitor thread
24/03/14 15:36:53 INFO YarnClientSchedulerBackend: Shutting down all executors
24/03/14 15:36:53 INFO YarnSchedulerBackend$YarnDriverEndpoint: Asking each executor to shut down
24/03/14 15:36:53 INFO YarnClientSchedulerBackend: YARN client scheduler backend Stopped
24/03/14 15:36:54 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/03/14 15:36:54 INFO MemoryStore: MemoryStore cleared
24/03/14 15:36:54 INFO BlockManager: BlockManager stopped
24/03/14 15:36:54 INFO BlockManagerMaster: BlockManagerMaster stopped
24/03/14 15:36:54 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/03/14 15:36:54 INFO SparkContext: Successfully stopped SparkContext
